In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# from sklearn.neighbors import NearestNeighbors
%matplotlib inline

In [2]:
df = pd.read_csv("ravdess_features.csv")

excluded = [
    "modality",
    # "vocal_channel",
    "emotional_intensity",
    "statement",
    "repetition",
    # "actor",
    "channels",
    "sample_width",
    "frame_rate",
    "frame_width",
    "sample_width",
    "stft_max",
    # "frame_count",
    # "length_ms",
    "mean",
    "max","min",
    # "kur","skew",
]
df.drop(columns=excluded, inplace=True)


df.actor.isna().sum()/len(df)

0.45921696574225124

Now missing intensity values are filled using $$I = 20 \; \log_{10}\left(\frac{\sigma}{\sigma_0}\right)$$

Then rows with missing actors are removed

In [3]:
import intensity_fill
df = intensity_fill.fill_intensity(df)

df_with_missing_actors = df.copy()

# df = df.dropna(subset=['actor'])

/home/djanloo/Desktop/dynamiting/intensity_fill.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.intensity[df.intensity.isna()] = I_from_std(df["std"][df.intensity.isna()])


In [4]:
for field in ["sex", "emotion", "vocal_channel"]:
    df[field] = pd.Categorical(df[field]).codes


The first missing feature that we try to fill is the `actor` field. We follow a knn clustering strategy

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram

cols = [col for col in df.columns if col != 'emotion']
X = df[cols].values

scaler = StandardScaler()
X_scal = scaler.fit_transform(X)
X_scal

scaler = MinMaxScaler()
X_minmax = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=8, n_init=10, max_iter=100)
kmeans.fit(X_minmax)

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
kmeans.labels_

array([5, 5, 0, ..., 1, 2, 1], dtype=int32)